# Model
Exicting part, try different types regression model.


In [12]:
import os
import io
import re
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
#read data, specify index so it's easier to join and search using loc
path = '../../data/new-york-city-airbnb-open-data/'
listings_csv = os.path.join(path,'model_columns_listings.csv')
listings = pd.read_csv(listings_csv)

In [3]:
#shape
listings.shape

(50540, 266)

In [4]:
listings.columns.values

array(['Unnamed: 0', 'id', 'host_since', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_listings_count', 'host_identity_verified', 'latitude',
       'longitude', 'is_location_exact', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'number_of_reviews',
       'number_of_reviews_ltm', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_per_month',
       'host_verifications_ facebook', 'host_verifications_ google',
       'host_verifications_ government_id',
       'host_verifications_ identity_manual', 'host_verifications_ jumio',
       'host_verifications_ kba', 'host_verifications_ manual_offline',
       'host_v

In [5]:
#head
listings.head(1)

,Unnamed: 0,id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_identity_verified,latitude,...,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
0,0,2060,733272,4.0,22%,50%,0,0.0,0,40.85722,...,0,0,0,0,0,0,0,0,0,1


# Features/Target
Features to be included and test/train split

In [143]:
#exclude for now
X = listings[['host_since', 'host_response_time', 'host_is_superhost', 'host_listings_count', 'host_identity_verified', 'latitude', 'longitude', 'is_location_exact', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'security_deposit', 'cleaning_fee', 'guests_included','extra_people', 'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'reviews_per_month', 'host_verifications_ facebook', 'host_verifications_ google', 'host_verifications_ government_id', 'host_verifications_ identity_manual', 'host_verifications_ jumio', 'host_verifications_ kba', 'host_verifications_ manual_offline', 'host_verifications_ manual_online', 'host_verifications_ offline_government_id', 'host_verifications_ phone', 'host_verifications_ reviews', 'host_verifications_ selfie', 'host_verifications_ sent_id', 'host_verifications_ sesame', 'host_verifications_ sesame_offline', 'host_verifications_ weibo', 'host_verifications_ work_email', 'host_verifications_ zhima_selfie', 'host_verifications_email', 'host_verifications_facebook', 'host_verifications_google', 'host_verifications_jumio', 'host_verifications_offline_government_id', 'host_verifications_phone', 'host_verifications_reviews', 'neighbourhood_group_cleansed_Bronx', 'neighbourhood_group_cleansed_Brooklyn', 'neighbourhood_group_cleansed_Manhattan', 'neighbourhood_group_cleansed_Queens', 'neighbourhood_group_cleansed_Staten Island', 'property_type_Aparthotel', 'property_type_Apartment','property_type_Barn', 'property_type_Bed and breakfast', 'property_type_Boat', 'property_type_Boutique hotel', 'property_type_Bungalow', 'property_type_Bus', 'property_type_Cabin', 'property_type_Camper/RV', 'property_type_Castle', 'property_type_Cave', 'property_type_Condominium', 'property_type_Cottage', 'property_type_Dome house', 'property_type_Dorm', 'property_type_Earth house', 'property_type_Farm stay', 'property_type_Guest suite', 'property_type_Guesthouse', 'property_type_Hostel', 'property_type_Hotel', 'property_type_House', 'property_type_Houseboat', 'property_type_In-law', 'property_type_Island', 'property_type_Lighthouse','property_type_Loft', 'property_type_Other', 'property_type_Resort', 'property_type_Serviced apartment', 'property_type_Tent', 'property_type_Timeshare', 'property_type_Tiny house', 'property_type_Townhouse', 'property_type_Train', 'property_type_Treehouse', 'property_type_Villa', 'property_type_Yurt', 'room_type_Entire home/apt', 'room_type_Hotel room', 'room_type_Private room', 'room_type_Shared room', 'cancellation_policy_flexible', 'cancellation_policy_moderate', 'cancellation_policy_strict', 'cancellation_policy_strict_14_with_grace_period', 'cancellation_policy_super_strict_30', 'cancellation_policy_super_strict_60', 'bed_type_Airbed', 'bed_type_Couch', 'bed_type_Futon','bed_type_Pull-out Sofa', 'bed_type_Real Bed', 'amenities_ toilet', 'amenities_24-hour check-in', 'amenities_Accessible-height bed','amenities_Accessible-height toilet', 'amenities_Air conditioning', 'amenities_Air purifier', 'amenities_BBQ grill', 'amenities_Baby bath', 'amenities_Baby monitor', 'amenities_Babysitter recommendations', 'amenities_Baking sheet', 'amenities_Barbecue utensils', 'amenities_Bath towel', 'amenities_Bathrobes', 'amenities_Bathroom essentials', 'amenities_Bathtub', 'amenities_Bathtub with bath chair', 'amenities_Beach essentials', 'amenities_Beachfront', 'amenities_Bed linens', 'amenities_Bedroom comforts', 'amenities_Bluetooth speaker', 'amenities_Body soap', 'amenities_Bottled water', 'amenities_Bread maker', 'amenities_Breakfast','amenities_Building staff', 'amenities_Buzzer/wireless intercom', 'amenities_Cable TV',
'amenities_Carbon monoxide detector', 'amenities_Cat(s)', 'amenities_Changing table', 'amenities_Children’s books and toys', 'amenities_Children’s dinnerware',
'amenities_Cleaning before checkout', 'amenities_Coffee maker',
'amenities_Cooking basics', 'amenities_Crib', 'amenities_Disabled parking spot',
'amenities_Dishes and silverware', 'amenities_Dishwasher', 'amenities_Dog(s)', 'amenities_Doorman', 'amenities_Dryer', 'amenities_EV charger', 'amenities_Electric profiling bed', 'amenities_Elevator', 'amenities_Essentials', 'amenities_Ethernet connection', 'amenities_Extra pillows and blankets', 'amenities_Extra space around bed', 'amenities_Family/kid friendly', 'amenities_Fire extinguisher', 'amenities_Fireplace guards', 'amenities_Firm mattress', 'amenities_First aid kit', 'amenities_Fixed grab bars for shower', 'amenities_Fixed grab bars for toilet', 'amenities_Flat path to guest entrance',
'amenities_Free parking on premises', 'amenities_Free street parking', 'amenities_Full kitchen', 'amenities_Game console', 'amenities_Garden or backyard', 'amenities_Ground floor access', 'amenities_Gym', 'amenities_Hair dryer', 'amenities_Handheld shower head', 'amenities_Hangers', 'amenities_Heating', 'amenities_High chair', 'amenities_Host greets you', 'amenities_Hot tub',
'amenities_Hot water', 'amenities_Hot water kettle', 'amenities_Indoor fireplace', 'amenities_Internet', 'amenities_Iron', 'amenities_Keypad', 'amenities_Kitchen',
'amenities_Kitchenette', 'amenities_Lake access', 'amenities_Laptop friendly workspace',
'amenities_Lock on bedroom door', 'amenities_Lockbox', 'amenities_Long term stays allowed',
'amenities_Luggage dropoff allowed', 'amenities_Microwave', 'amenities_Mini bar', 'amenities_Mini fridge', 'amenities_Mobile hoist', 'amenities_No stairs or steps to enter', 'amenities_Other', 'amenities_Other pet(s)', 'amenities_Outlet covers', 'amenities_Oven', 'amenities_Pack ’n Play/travel crib', 'amenities_Paid parking off premises', 'amenities_Paid parking on premises', 'amenities_Patio or balcony', 'amenities_Pets allowed', 'amenities_Pets live on this property', 'amenities_Pocket wifi', 'amenities_Pool', 'amenities_Private bathroom', 'amenities_Private entrance', 'amenities_Private living room', 'amenities_Record player', 'amenities_Refrigerator', 'amenities_Room-darkening shades', 'amenities_Safe', 'amenities_Safety card', 'amenities_Self check-in', 'amenities_Shampoo', 'amenities_Shower chair', 'amenities_Shower gel',
'amenities_Single level home', 'amenities_Ski-in/Ski-out', 'amenities_Slippers', 'amenities_Smart lock', 'amenities_Smoke detector', 'amenities_Smoking allowed', 'amenities_Snacks', 'amenities_Stair gates', 'amenities_Step-free shower', 'amenities_Stove',
'amenities_Suitable for events', 'amenities_TV', 'amenities_Table corner guards', 'amenities_Toilet paper', 'amenities_Trash can', 'amenities_Turndown service',
'amenities_Washer', 'amenities_Washer/Dryer', 'amenities_Waterfront', 'amenities_Well-lit path to entrance', 'amenities_Wheelchair accessible', 'amenities_Wide clearance to shower',
'amenities_Wide doorway to guest bathroom', 'amenities_Wide entrance', 'amenities_Wide entrance for guests', 'amenities_Wide entryway', 'amenities_Wide hallways', 'amenities_Wifi', 'amenities_Window guards']]


y = listings['price']

# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Lasso Regression

In [144]:
#using CV
regressor = Lasso(alpha=0.5)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)

print(rmse_scores)


[142.67975614 122.34259414 118.58493709 136.7941142  118.37828297
 125.10905294 108.94748914 124.29553557 129.77773808 117.26292861]


In [145]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))


124.41724288715648
0.3825604491710666


In [146]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

123.3710945532224
0.3641275009215623


# Ridge Regression

In [10]:
#using CV
regressor = Ridge(alpha=0.5)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)
print(rmse_scores)


[140.42168081 120.54958581 115.75151057 134.84566413 116.50249749
 122.18631812 106.66643248 122.50434007 128.88242786 115.79063476]


In [11]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))


122.41010921044267
0.4078166023213534


In [12]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

121.44489517482583
0.38382831737510925


# Random Forest

In [8]:
#using CV
regressor = RandomForestRegressor(n_estimators=100, random_state=None, min_samples_split=100)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)
print(rmse_scores)

[132.94518643 112.50746338 109.71129262 125.89160636 108.1108448
 113.24647674  98.51975492 113.86064691 121.08207248 107.72720211]


In [9]:
pred_train= regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

114.36025467589324
0.5894403114339299


In [10]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

114.21896939911265
0.4549709116973968


# Feature Importance for Random Forest

In [67]:
#get features importance
features = regressor.feature_importances_

#get column names from our list of features
column_names=X.columns.values.tolist()

#match features importance to column name
df = pd.DataFrame([features], columns=column_names)


In [84]:
df.head()

,host_since,host_response_time,host_is_superhost,host_listings_count,host_identity_verified,latitude,longitude,is_location_exact,accommodates,bathrooms,...,amenities_Well-lit path to entrance,amenities_Wheelchair accessible,amenities_Wide clearance to shower,amenities_Wide doorway to guest bathroom,amenities_Wide entrance,amenities_Wide entrance for guests,amenities_Wide entryway,amenities_Wide hallways,amenities_Wifi,amenities_Window guards
0,0.0155,0.003355,0.000485,0.006602,0.000118,0.04189,0.07062,0.000146,0.248767,0.178836,...,0.000053,0.00217,0.000013,0.000025,0.000019,0.000012,4.199719e-07,0.000074,0.002462,0.00016


In [106]:
#features importance sorted
features_sorted = df.iloc[0].sort_values()

In [125]:
#unused features
filt = features_sorted.values > features_sorted.mean()
features_sorted[filt]

amenities_Essentials                      0.004768
amenities_Doorman                         0.005060
amenities_24-hour check-in                0.005338
amenities_Paid parking on premises        0.005742
room_type_Hotel room                      0.006157
guests_included                           0.006298
property_type_Apartment                   0.006334
host_listings_count                       0.006602
room_type_Private room                    0.008467
reviews_per_month                         0.008588
property_type_Boutique hotel              0.008829
extra_people                              0.009234
property_type_Resort                      0.012871
host_since                                0.015500
security_deposit                          0.028335
cleaning_fee                              0.031611
bedrooms                                  0.032177
latitude                                  0.041890
neighbourhood_group_cleansed_Manhattan    0.052741
longitude                      

In [134]:
X = listings[['amenities_Essentials','amenities_Doorman','amenities_24-hour check-in','amenities_Paid parking on premises','room_type_Hotel room','guests_included','property_type_Apartment','host_listings_count','room_type_Private room','reviews_per_month','property_type_Boutique hotel','extra_people','property_type_Resort','host_since','security_deposit','cleaning_fee','bedrooms','latitude','neighbourhood_group_cleansed_Manhattan','longitude','room_type_Entire home/apt','bathrooms','accommodates']]

In [135]:
#rerun using featuers that are heavily used
regressor = RandomForestRegressor(n_estimators=100, random_state=None, min_samples_split=100)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)
print(rmse_scores)

[132.55367644 112.52835586 109.95504707 125.62918321 107.60999167
 113.76936454  97.99807738 114.30104226 121.22616281 107.41710145]


In [136]:
pred_train= regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

114.29880026773921
0.5902263314769265


In [137]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

114.5252089822505
0.4520443709069428
